# **TextTorch - 03: Definição do Modelo**

**Objetivo:** Definir a arquitetura da rede neural que será usada para a classificação. Este notebook carrega os dados processados e instancia o modelo PyTorch conforme a configuração.

### **Como Executar (Google Colab)**

1. **Pré-requisito:** Execute os notebooks `01_preprocessing.ipynb` e `02_representation.ipynb`.
2. **Ambiente:** Se estiver em um novo ambiente, clone o repositório e instale as dependências.
   ```bash
   !git clone https://github.com/takaokensei/TextTorch.git
   %cd TextTorch
   !pip install -r requirements.txt
   ```
3. **Execução:** Execute todas as células deste notebook.

In [1]:
# Imports e Configurações Iniciais
import sys
import os
import torch

# Adiciona o diretório 'src' ao path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from model import create_model, load_config

# Carrega configurações
CONFIG_PATH = '../models/config.yaml'
config = load_config(CONFIG_PATH)

# Carrega tensores processados
TENSORS_PATH = f"../artifacts/tensors_{config['representation']}.pt"
data = torch.load(TENSORS_PATH)

print(f"Configuração e tensores para '{config['representation']}' carregados.")

2025-11-13 17:28:59,720 - INFO - Configuração carregada de: ../models/config.yaml


Configuração e tensores para 'tfidf' carregados.


## 1. Arquitetura do Modelo (TF-IDF)

Para a representação TF-IDF, usamos um classificador feedforward simples. A arquitetura é:

1.  **Camada de Entrada:** `Linear(input_dim, hidden_dim)`
    - `input_dim`: Tamanho do vocabulário TF-IDF (ex: 10.000).
    - `hidden_dim`: Dimensão da camada oculta (ex: 512).
2.  **Função de Ativação:** `ReLU`
3.  **Regularização:** `Dropout` (taxa de 0.5 para evitar overfitting).
4.  **Camada de Saída:** `Linear(hidden_dim, n_classes)`
    - `n_classes`: Número de classes a serem preditas.

In [2]:
# Extrai dimensões dos dados carregados
input_dim = data['input_dim']
n_classes = data['n_classes']

# Cria o modelo usando a factory function
model = create_model(input_dim=input_dim, n_classes=n_classes, config=config)

# Exibe a arquitetura do modelo
print("--- Arquitetura do Modelo ---")
print(model)

# Exibe o número de parâmetros treináveis
print(f"\nNúmero total de parâmetros treináveis: {model.count_parameters():,}")

2025-11-13 17:29:06,215 - INFO - TFIDFClassifier criado: input=676, hidden=512, output=6


--- Arquitetura do Modelo ---
TFIDFClassifier(
  (fc1): Linear(in_features=676, out_features=512, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=6, bias=True)
)

Número total de parâmetros treináveis: 349,702


## 2. Teste de Forward Pass

Para garantir que o modelo está corretamente configurado, realizamos um "forward pass" com um pequeno batch de dados. Isso verifica se as dimensões dos tensores estão alinhadas e se o modelo produz uma saída com o formato esperado (`batch_size`, `n_classes`).

In [3]:
print("--- Testando Forward Pass ---")

# Pega um pequeno batch do conjunto de treino
X_sample = data['X_train'][:4]  # Batch de 4 amostras

print(f"Shape do tensor de entrada: {X_sample.shape}")

# Realiza o forward pass
try:
    model.eval()  # Coloca o modelo em modo de avaliação
    with torch.no_grad():
        outputs = model(X_sample)
    
    print(f"Shape do tensor de saída: {outputs.shape}")
    assert outputs.shape == (4, n_classes)
    print("\nForward pass concluído com sucesso!")
    
except Exception as e:
    print(f"\nERRO durante o forward pass: {e}")

--- Testando Forward Pass ---
Shape do tensor de entrada: torch.Size([4, 676])
Shape do tensor de saída: torch.Size([4, 6])

Forward pass concluído com sucesso!


---

## **OPCIONAL: Arquitetura com Embeddings**

Se `representation` em `config.yaml` for `embedding`, o código abaixo definirá um modelo com uma camada de embedding e um LSTM.

**Arquitetura:**
1.  **Embedding:** `nn.Embedding(vocab_size, embedding_dim)`
2.  **Recorrente:** `LSTM(embedding_dim, hidden_dim, bidirectional=True)`
3.  **Regularização:** `Dropout`
4.  **Camada de Saída:** `Linear(hidden_dim * 2, n_classes)` (x2 por ser bidirecional)

In [ ]:
# --- CÓDIGO PARA EMBEDDINGS (EXECUTADO SE CONFIGURADO) ---

# if config['representation'] == 'embedding':
#     # Carrega os dados específicos de embedding
#     TENSORS_EMB_PATH = '../artifacts/tensors_embedding.pt'
#     data_emb = torch.load(TENSORS_EMB_PATH)
    
#     # Descomente a classe EmbeddingClassifier em src/model.py para usar
#     try:
#         model_emb = create_model(
#             input_dim=data_emb['vocab_size'], # Aqui, input_dim é vocab_size
#             n_classes=data_emb['n_classes'], 
#             config=config
#         )
        
#         print("\n--- Arquitetura do Modelo de Embedding ---")
#         print(model_emb)
#         print(f"\nParâmetros treináveis: {model_emb.count_parameters():,}")
        
#     except NotImplementedError as e:
#         print(f"\nAVISO: {e}")